In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Intest Rate Swap with fixed and floating leg

Demonstrates creation of an interest rate swap in LUSID with a 
fixed and floating rate leg.

Attributes
----------
interest rate swap
instruments
"""

toggle_code("Hide docstring")

/opt/conda/lib/python3.9/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## Modelling a swap in LUSID using LUSID instrument model

In [2]:
# Python imports
import os
from datetime import datetime, timedelta
import pytz
import json

# Lusid imports
import lusid
from lusid import models
from lusid.utilities import ApiClientFactory
from lusid.models import flow_conventions
from lusidjam import RefreshingToken 

# comment below for prod
secrets_path = os.getenv("FBN_SECRETS_PATH") 
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

# Setup the apis we'll use in this notebook:
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
    

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)


LUSID Environment Initialised
LUSID SDK Version:  0.6.9157.0


In [3]:
#Dates, scopes and market data
start = datetime(2022, 1, 4, tzinfo=pytz.utc)
maturity = datetime(2023, 1, 4, tzinfo=pytz.utc)
scope = "ExampleIRSUpsert"
ca_code = "ibor_corp_act"
portfolio_code = "SwapPortfolio"



## (1) Defining the full instrument model

In [4]:
## Define flow convention for Fixed LEg

flow_conventionFixed = models.FlowConventions(
    currency="USD",
    payment_frequency="1Y",
    day_count_convention="Act360",
    roll_convention="ModifiedFollowing",
    settle_days=0,
    reset_days=0,
    payment_calendars=[],
    reset_calendars=[]
)

## Define flow convention for floating leg

flow_conventionFloat = models.FlowConventions(
    currency="USD",
    payment_frequency="1Y",
    day_count_convention="Act360",
    roll_convention="ModifiedFollowing",
    settle_days=0,
    reset_days=0,
    payment_calendars=[],
    reset_calendars=[]
)

# For projection rates it requires:
# ccy/tenor/indexname dependency
# For past data:
# Looks for quotes in quote store with reference = fixingreference
float_leg_idx_conv = models.IndexConvention(
    currency="USD",
    payment_tenor="1D",
    fixing_reference="USD.1D.SOFRFIXING", # this links to instrument_id in quoteseriesID
    publication_day_lag=0,
    day_count_convention="Act360",
    index_name="SOFR"
)

# Create the leg definitions
float_leg_definition = models.LegDefinition(
    index_convention=float_leg_idx_conv,
    pay_receive="Receive",
    conventions=flow_conventionFloat,
    stub_type="Both",
    notional_exchange_type="None",
    rate_or_spread=0.0025,
    reset_convention="InArrears",
    compounding = ""
)

fixed_leg_definition = models.LegDefinition(
    rate_or_spread=0.025,
    pay_receive="Pay",
    conventions=flow_conventionFixed,
    stub_type="Both",
    notional_exchange_type="None",
)

# Create the legs
fixed_leg = models.FixedLeg(
    start_date=start.isoformat(),
    maturity_date=maturity.isoformat(),
    notional=10000000,
    leg_definition=fixed_leg_definition,
    instrument_type="FixedLeg",
    overrides={}
)

floating_leg = models.FloatingLeg(
    start_date=start.isoformat(),
    maturity_date=maturity.isoformat(),
    notional=10000000,
    leg_definition=float_leg_definition,
    instrument_type="FloatingLeg",
    overrides={}
)

irs_legs = [
    fixed_leg,
    floating_leg
]

# create the swap
instrument_definition_IRS = models.InterestRateSwap(
    start_date=start.isoformat(),
    maturity_date=maturity.isoformat(),
    legs=irs_legs,
    instrument_type="InterestRateSwap")



## (2) Create property defintions for leg ID's

In [5]:
try:
    property_definition_request = models.CreatePropertyDefinitionRequest(
        domain="Instrument",
        scope=scope,
        code="payLegId",
        display_name="pay LegId",
        data_type_id=models.ResourceId(
            scope="system",
            code="number",
        ),
        life_time="Perpetual",
    )

    property_definitions_api.create_property_definition(
        create_property_definition_request=property_definition_request
    )
    property_definition_request2 = models.CreatePropertyDefinitionRequest(
        domain="Instrument",
        scope=scope,
        code="receiveLegID",
        display_name="receive LegID",
        data_type_id=models.ResourceId(
            scope="system",
            code="number",
        ),
        life_time="Perpetual",
    )

    property_definitions_api.create_property_definition(
        create_property_definition_request=property_definition_request2
    )
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

Error creating Property Definition 'Instrument/ExampleIRSUpsert/payLegId' because it already exists.


## (3) Modelling as simple instrument

In [6]:
simpleIRS = models.SimpleInstrument(
        instrument_type="SimpleInstrument",
        dom_ccy="USD",
        asset_class="InterestRates",
        simple_instrument_type="InterestRates",
    )

payLegIdpropertySimple = models.ModelProperty(
    key=f"Instrument/{scope}/payLegId",
    value=models.PropertyValue(
        metric_value=models.MetricValue(
            value=99998,
            #unit="Decimal",
        )
    ),
)

receiveLegIDpropertySimple = models.ModelProperty(
    key=f"Instrument/{scope}/receiveLegID",
    value=models.PropertyValue(
        metric_value=models.MetricValue(
            value=99997,
            #unit="Decimal",
        )
    ),
)

payLegIdproperty = models.ModelProperty(
    key=f"Instrument/{scope}/payLegId",
    value=models.PropertyValue(
        metric_value=models.MetricValue(
            value=12345,
            #unit="Decimal",
        )
    ),
)

receiveLegIDproperty = models.ModelProperty(
    key=f"Instrument/{scope}/receiveLegID",
    value=models.PropertyValue(
        metric_value=models.MetricValue(
            value=12346,
            #unit="Decimal",
        )
    ),
)

swap_definition = models.InstrumentDefinition(
    name="MySimpleIRS",
    identifiers={"ClientInternal": models.InstrumentIdValue("Swap1")},
    definition=simpleIRS,
    properties=[payLegIdpropertySimple,receiveLegIDpropertySimple],
)

IRS_definition = models.InstrumentDefinition(
    name="InterestRateSwap",
    identifiers={"ClientInternal": models.InstrumentIdValue("Swap2")},
    definition=instrument_definition_IRS,
    properties=[payLegIdproperty,receiveLegIDproperty],
)

# upsert the instrument
upsert_request = {"Swap1": swap_definition, "Swap2" : IRS_definition}
upsert_response = instruments_api.upsert_instruments(request_body=upsert_request, scope=scope)


## (4) Query swap using individual leg ID's

In [7]:
legfilter = f"properties[Instrument/{scope}/payLegId] eq 99998"
listofinstruments = instruments_api.list_instruments(scope=scope, filter=legfilter, instrument_property_keys= [f"Instrument/{scope}/payLegId",f"Instrument/{scope}/receiveLegID"])
print(listofinstruments.values[0].properties)

[{'effective_from': datetime.datetime(1, 1, 1, 0, 0, tzinfo=tzlocal()),
 'effective_until': datetime.datetime(9999, 12, 31, 23, 59, 59, 999999, tzinfo=tzlocal()),
 'key': 'Instrument/ExampleIRSUpsert/payLegId',
 'value': {'label_value': None,
           'label_value_set': None,
           'metric_value': {'unit': None, 'value': 99998.0}}}, {'effective_from': datetime.datetime(1, 1, 1, 0, 0, tzinfo=tzlocal()),
 'effective_until': datetime.datetime(9999, 12, 31, 23, 59, 59, 999999, tzinfo=tzlocal()),
 'key': 'Instrument/ExampleIRSUpsert/receiveLegID',
 'value': {'label_value': None,
           'label_value_set': None,
           'metric_value': {'unit': None, 'value': 99997.0}}}]


In [8]:
legfilter = f"properties[Instrument/{scope}/payLegId] eq 12345"
listofinstruments = instruments_api.list_instruments(scope=scope, filter=legfilter, instrument_property_keys= [f"Instrument/{scope}/payLegId",f"Instrument/{scope}/receiveLegID"])
print(listofinstruments.values[0].properties)

[{'effective_from': datetime.datetime(1, 1, 1, 0, 0, tzinfo=tzlocal()),
 'effective_until': datetime.datetime(9999, 12, 31, 23, 59, 59, 999999, tzinfo=tzlocal()),
 'key': 'Instrument/ExampleIRSUpsert/payLegId',
 'value': {'label_value': None,
           'label_value_set': None,
           'metric_value': {'unit': None, 'value': 12345.0}}}, {'effective_from': datetime.datetime(1, 1, 1, 0, 0, tzinfo=tzlocal()),
 'effective_until': datetime.datetime(9999, 12, 31, 23, 59, 59, 999999, tzinfo=tzlocal()),
 'key': 'Instrument/ExampleIRSUpsert/receiveLegID',
 'value': {'label_value': None,
           'label_value_set': None,
           'metric_value': {'unit': None, 'value': 12346.0}}}]
